In [ ]:
import random
import numpy as np
from PIL import Image, ImageDraw,ImageFont
import os
import math
from shutil import rmtree
import time

In [ ]:
def do_point_map(): #Создания случайных точек на плоскости
  pt=0
  if(PointFlag==1):
    pt=2
  for now in range(pt,nCity):
    thisnear=[]
    point_x.append(random.randint(1, window_x-2))
    point_y.append(random.randint(1, window_y-2))
    for other in range(now):
      dist=((point_x[other]-point_x[now])**2 + (point_y[other]-point_y[now])**2)**0.5
      if (dist<near_range):
        thisnear.append(other)
        point_near[other].append(now)
        lenTrail[now][other]=dist
        lenTrail[other][now]=dist
    point_near.append(thisnear)

In [ ]:
def FirstDraw(): # Рисование изначального построенного графа
  image = Image.new("RGB", (window_x*ScrCoef,window_y*ScrCoef), (255,255,255))
  draw = ImageDraw.Draw(image)
  for now in range(nCity):
    for other in range(len(point_near[now])):
      if (point_near[now][other]<now):
        draw.line([(ScrCoef*point_x[point_near[now][other]],ScrCoef*point_y[point_near[now][other]]),(ScrCoef*point_x[now],ScrCoef*point_y[now])], fill ="black", width = 0) 
  for now in range(nCity):
    if(now==start or now==end):
      draw.ellipse([(ScrCoef*point_x[now]-5,ScrCoef*point_y[now]-5),(ScrCoef*point_x[now]+5,ScrCoef*point_y[now]+5)], fill = 'blue', outline ='blue')
      draw.text((ScrCoef*point_x[now]+5,ScrCoef*point_y[now]+5), str(now),fill='blue')
    else:
      draw.ellipse([(ScrCoef*point_x[now]-2,ScrCoef*point_y[now]-2),(ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2)], fill = 'red', outline ='red')
      draw.text((ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2), str(now),fill='red')
  del draw
  image.save("Map.png", "PNG")

In [ ]:
def ProgressDraw(time,bestLen,Trail,FoldName): #Рисование карты - промежуточно найденного лучшего пути
  image = Image.new("RGB", (ScrCoef*window_x,ScrCoef*window_y), (255,255,255))
  draw = ImageDraw.Draw(image)
  for i in range(len(Trail)-1):
     draw.line([(ScrCoef*point_x[Trail[i]],ScrCoef*point_y[Trail[i]]),(ScrCoef*point_x[Trail[i+1]],ScrCoef*point_y[Trail[i+1]])], fill ="red", width = 5)
  for now in range(nCity):
    for other in range(len(point_near[now])):
      otherCit=point_near[now][other]
      if (otherCit<now):
        draw.line([(ScrCoef*point_x[otherCit],ScrCoef*point_y[otherCit]),(ScrCoef*point_x[now],ScrCoef*point_y[now])], fill ="black", width = 0) 
        draw.text((ScrCoef*(point_x[otherCit]+point_x[now])/2-2,ScrCoef*(point_y[otherCit]+point_y[now])/2-2), str('%.3f' % pherTrail[now][otherCit]),fill='blue')
  for now in range(nCity):
    draw.ellipse([(ScrCoef*point_x[now]-2,ScrCoef*point_y[now]-2),(ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2)], fill = 'black', outline ='black')
  draw.ellipse([(ScrCoef*point_x[start]-4,ScrCoef*point_y[start]-4),(ScrCoef*point_x[start]+4,ScrCoef*point_y[start]+4)], fill = 'blue', outline ='blue')
  draw.ellipse([(ScrCoef*point_x[end]-4,ScrCoef*point_y[end]-4),(ScrCoef*point_x[end]+4,ScrCoef*point_y[end]+4)], fill = 'red', outline ='red')
  del draw
  name=FoldName+"/"+"time"+str(time)+"best"+str(bestLen)+".png"
  image.save(name, "PNG")

In [ ]:
def DeikstraDraw(Trail,FileName): #Рисование карты - промежуточно найденного лучшего пути
  image = Image.new("RGB", (ScrCoef*window_x,ScrCoef*window_y), (255,255,255))
  draw = ImageDraw.Draw(image)
  for i in range(len(Trail)-1):
     draw.line([(ScrCoef*point_x[Trail[i]],ScrCoef*point_y[Trail[i]]),(ScrCoef*point_x[Trail[i+1]],ScrCoef*point_y[Trail[i+1]])], fill ="red", width = 5)
  for now in range(nCity):
    for other in range(len(point_near[now])):
      otherCit=point_near[now][other]
      if (otherCit<now):
        draw.line([(ScrCoef*point_x[otherCit],ScrCoef*point_y[otherCit]),(ScrCoef*point_x[now],ScrCoef*point_y[now])], fill ="black", width = 0) 
  for now in range(nCity):
    draw.ellipse([(ScrCoef*point_x[now]-2,ScrCoef*point_y[now]-2),(ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2)], fill = 'black', outline ='black')
  draw.ellipse([(ScrCoef*point_x[start]-4,ScrCoef*point_y[start]-4),(ScrCoef*point_x[start]+4,ScrCoef*point_y[start]+4)], fill = 'blue', outline ='blue')
  draw.ellipse([(ScrCoef*point_x[end]-4,ScrCoef*point_y[end]-4),(ScrCoef*point_x[end]+4,ScrCoef*point_y[end]+4)], fill = 'red', outline ='red')
  del draw
  name=FileName+".png"
  image.save(name, "PNG")

In [ ]:
def probability(i,j,to): #Расчёт весов определённого отрезка пути для расчёта вероятности прохода муравья по нему.
  p=(pherTrail[i][j]**Alpha)*((1/lenTrail[i][j])**Beta)
  return p

In [ ]:
def UpdPher(): # Обновление феромонов
  for i in range(nCity):
    for j in range (len(point_near[i])):
      nj=point_near[i][j]
      if(lenTrail[i][nj]!=0):
        pherTrail[i][nj]=pherTrail[i][nj]*(1-Rho)+deltaPher[i][nj]
        if(pherTrail[i][nj]<pherMin):
          pherTrail[i][nj]=pherMin
        if(pherTrail[i][nj]>pherMax):
          pherTrail[i][nj]=pherMax
        pherTrail[nj][i]=pherTrail[i][nj]
        deltaPher[i][nj]=0
        deltaPher[nj][i]=0

In [ ]:
def Move(now,tabu,to): #Функция выбора следующей точки для перехода
  WhereList=[]
  ProbList=[]
  ErrFlag=1
  for cit in range(len(point_near[now])):
    near=point_near[now][cit]
    if((lenTrail[now][near]==0) or (tabu.count(near)!=0)):
      continue
    else:
      ErrFlag=0
      WhereList.append(near)
      ProbList.append(probability(now,near,to))
  if (ErrFlag==1):
    return -1 
  return random.choices(WhereList, weights=ProbList)[0]

In [ ]:
def March(Fr,To): #Функция перехода муравья из одной точки в другую
  now=Fr
  tabu=[]
  tabu.append(now)
  while 1:
    next=Move(now,tabu,To)
    if (next==-1):
      tabu.clear()
      now=Fr
      tabu.append(now)
    else:
      now=next
      tabu.append(now)
      if (now==To):
        return tabu

In [ ]:
def Length(Trail): #Расчёт длины пути на графе
  l=0
  for i in range(len(Trail)-1):
    l+=lenTrail[Trail[i]][Trail[i+1]]
  return l

In [ ]:
def PutPher(Trail,l): #Распределение феромонов после движения муравьёв
  for sch in range(len(Trail)-1):
    deltaPher[Trail[sch]][Trail[sch+1]]+=Q/l
    deltaPher[Trail[sch+1]][Trail[sch]]=deltaPher[Trail[sch]][Trail[sch+1]]

In [ ]:
nAnt=10 #Количество простых муравьёв
Tend=1000 #Время симуляции
Alpha=1 #Коэффициент альфа (порядка значимости феромона)
Beta=1 #Коэффициент бэта (порядка значимости длины пути)
Rho=0.2 #Коэффициент испарения феромона
Q=400 #Коэффициент увеличения феромона

pherMin=1 #Минимальное количество феромона на рёбрах графа
pherMax=10 #Максимальное количество феромона на рёбрах графа

In [ ]:
point_x=[]
point_y=[]
point_near=[]

window_x=400 #ширина поля
window_y=400 #высота поля
ScrCoef=4 #Масштабирование при выводе карт

PointFlag=1 #Поставить 1, чтоб точки старта и конца были по углам иначе 0. Можно установить также позже изменив start и end
MapMode=0 #Поставить 1, чтоб составить карту из точек в виде сетки, 2 если разлиновать готовую карту. Иначе случайная расстановка точек.

start=0 #Точка муравейника
end=1 #Точка цели

nCity=100 #Количество вершин

if (PointFlag==1):
  point_x.append(20)
  point_y.append(20)
  point_x.append(window_x-20)
  point_y.append(window_y-20)
  nCity+=2
  thisnear1=[]
  point_near.append(thisnear1)
  thisnear2=[]
  point_near.append(thisnear2)
  start=0 #Точка муравейника
  end=1 #Точка цели

if (MapMode==1):
  near_range=50
  nCity,lenTrail=do_point_map_cage()
  FirstDraw()
elif (MapMode==2):
  near_range=30
  adress="/content/drive/My Drive/Безымянный.png"
  block_display=0 #1 чтобы включить отображение препятствий на карте. Иначе выключить
  window_x,window_y,nCity,lenTrail=do_point_map_cage_im(adress)
  start=0 #Точка муравейника
  end=542 #Точка цели
  FirstDrawIm(adress,block_display)
else:
  near_range=2*(window_x*window_y/nCity)**0.5 #Радиус, внутри которого точки считаются соседствующими
  lenTrail = [[0 for j in range(nCity)] for i in range(nCity)]
  do_point_map()
  FirstDraw()

#FirstDraw() #Сохранение рисунка карты


In [ ]:
# Модификации
nElite=0 #Количество элитных муравьёв
DirModif=0 #Добавление в вероятность выбора множитель направления

In [ ]:
# Имена и циклы
dir_name="Tests"
# Первая переменная цикла (Alpha, Beta, Rho, Q, nElite)
Par1Name="Alpha"
Rng1max = 0
# Вторая переменная цикла
Par2Name="Beta"
Rng2max = 0
# Третья переменная цикла
Par3Name="Q"
Rng3max = 0

MaxCikl=1

ShowProcessMaps=1 # Отображение маршрутов на картах: 0 - нет, 1 - да
ShowProgressString=1 # Отображение строк с нахождением новых оптимальных путей: 0 - нет, 1 - да

In [ ]:
#Цикл для исследования ACO с различными параметрами
if (os.path.exists(dir_name)):
  rmtree(dir_name)
if (ShowProcessMaps==1):
  os.mkdir(dir_name)

BestList=[]
TimerList=[]
StepList=[]
MedianList=[]

for Par1 in range(Rng1max+1):
  #Alpha=Par1
  
  BestListPar1=[]
  TimerListPar1=[]
  StepListPar1=[]
  MedianListPar1=[]
  
  if(ShowProcessMaps==1):
    dir_name1=dir_name+"/"+Par1Name+"_"+str(Par1)
    os.mkdir(dir_name1)
  for Par2 in range(Rng2max+1):
    #Beta=Par2

    BestListPar2=[]
    TimerListPar2=[]
    StepListPar2=[]
    MedianListPar2=[]

    if(ShowProcessMaps==1):
      dir_name2=dir_name1+"/"+Par2Name+"_"+str(Par2)
      os.mkdir(dir_name2)

    for Par3 in range(Rng3max+1):
      #Q=200+Par3*200

      BestListPar3=[]
      TimerListPar3=[]
      StepListPar3=[]

      if(ShowProcessMaps==1):
        dir_name3=dir_name2+"/"+Par3Name+"_"+str(Par3)
        os.mkdir(dir_name3)
      for Cikl in range(MaxCikl):
        BestListCikl=[]
        TimerListCikl=[]
        StepListCikl=[]
        RealStartTime = time.time()

        if(ShowProcessMaps==1):
          dir_name4=dir_name3+"/"+"Cikl"+"_"+str(Cikl+1)
          os.mkdir(dir_name4)
        pherTrail = [[pherMin for j in range(nCity)] for i in range(nCity)]
        deltaPher = [[0 for j in range(nCity)] for i in range(nCity)]
        bestLen=-1
        bestTrail=[]
        bestTime=0
        for step in range(Tend):
          for Ant in range(nAnt):
            if(step%2==0):
              way=March(start,end)
            else:
              way=March(end,start)
            l=Length(way)
            if(bestLen==-1 or l<bestLen):
              bestLen=l
              bestTrail=way
              bestTime=step
              
              RealNowTime = time.time()
              BestListCikl.append(bestLen)
              TimerListCikl.append(RealNowTime-RealStartTime)
              StepListCikl.append(step)
              if(ShowProgressString==1):
                print("На шаге",step," (",round(RealNowTime-RealStartTime,4),"сек) найден путь длиной",bestLen)
              if(ShowProcessMaps==1):
                ProgressDraw(step,round(bestLen,1),bestTrail,dir_name4)
            
            PutPher(way,l)
          for EliteAnt in range(nElite):
            PutPher(bestTrail,bestLen)
          UpdPher()
        if(ShowProgressString==1):
          print("len",bestLen)
          print("step",bestTime)
          print("time",round(RealNowTime-RealStartTime,4))
        print("Alpha = "+str(Alpha))
        print("Beta = "+str(Beta))
        print("Par3 = "+str(Par3))
        print("Cikl = "+str(Cikl))

        BestListPar3.append(BestListCikl) #Собираем в список с каждого цикла
        TimerListPar3.append(TimerListCikl)
        StepListPar3.append(StepListCikl)
      
      copyBest=BestListPar3.copy()
      copyBest.sort()
      Median=copyBest[MaxCikl//2]
      for k in range (MaxCikl):
        if(BestListPar3[k]==Median):
          MedianNum=k
          break
      MedianListPar2.append(MedianNum) #Собираем в список разных параметров 3
      BestListPar2.append(BestListPar3) 
      TimerListPar2.append(TimerListPar3)
      StepListPar2.append(StepListPar3)
    MedianListPar1.append(MedianListPar2) #Собираем в список разных параметров 2
    BestListPar1.append(BestListPar2) 
    TimerListPar1.append(TimerListPar2)
    StepListPar1.append(StepListPar2)
  MedianList.append(MedianListPar1) #Собираем в список разных параметров 1
  BestList.append(BestListPar1)
  TimerList.append(TimerListPar1)
  StepList.append(StepListPar1)


На шаге 0  ( 0.0062 сек) найден путь длиной 3650.7301908575464
На шаге 0  ( 0.1947 сек) найден путь длиной 3213.2926147894887
На шаге 0  ( 0.381 сек) найден путь длиной 2241.104581229008
На шаге 0  ( 0.5705 сек) найден путь длиной 1621.7460297253024
На шаге 0  ( 0.7631 сек) найден путь длиной 1171.5066701675712
На шаге 1  ( 0.9779 сек) найден путь длиной 908.7444484815809
На шаге 8  ( 1.3436 сек) найден путь длиной 890.2433491935567
На шаге 9  ( 1.5432 сек) найден путь длиной 784.8739715769775
На шаге 74  ( 3.6346 сек) найден путь длиной 681.3015748871655
На шаге 609  ( 18.9641 сек) найден путь длиной 618.9397505301141
На шаге 979  ( 29.4995 сек) найден путь длиной 600.0704918417833
len 600.0704918417833
step 979
time 29.4995
Alpha = 1
Beta = 1
Par3 = 0
Cikl = 0


In [ ]:
def Deikstra(Now):
  for Look in (point_near[Now]):
    if(lenTrail[Now][Look]+MinRouteLen[Now]<MinRouteLen[Look]):
      MinRouteLen[Look]=lenTrail[Now][Look]+MinRouteLen[Now]
      flag=0
      for i in (NotBlocked):
        if(i==Look):
          flag=1
          break
      if (flag==0):
        NotBlocked.append(Look)
      MinRoute[Look]=MinRoute[Now].copy()
      MinRoute[Look].append(Look)
  NotBlocked.remove(Now)

In [ ]:
#Алгоритмы поиска пути на взвешенном графе Дейкстра
RealStartTime = time.time()

NotBlocked=[]
NotBlocked.append(start)
MinRouteLen=[]
for i in range(nCity):
  MinRouteLen.append(10000000)
MinRoute=[]
for j in range(nCity):
  OnePoint=[]
  MinRoute.append(OnePoint)
MinRoute[start].append(start)
MinRouteLen[start]=0
Deikstra(start)
while(1):
  if(len(NotBlocked)==0):
    break
  min=NotBlocked[0]
  for k in (NotBlocked):
    if(MinRouteLen[min]>MinRouteLen[k]):
      min=k
  Deikstra(min)

RealNowTime = time.time()
DeltaTime=RealNowTime-RealStartTime

print("Time:"+str(DeltaTime))
print("Best route:"+str(MinRoute[end]))
print("Length of best route:"+str(MinRouteLen[end]))
DeikstraDraw(MinRoute[end],"Deikstra")

minDei=MinRouteLen[end]

Time:0.002285480499267578
Best route:[0, 95, 59, 14, 94, 7, 22, 43, 26, 1]
Length of best route:538.8033612520721


In [ ]:
import openpyxl

# создаем новый excel-файл
wb = openpyxl.Workbook()

# добавляем новый лист
wb.create_sheet(title = 'Первый лист', index = 0)

# получаем лист, с которым будем работать
sheet = wb['Первый лист']


for i in range (len(BestList)):
  for j in range (len(BestList[i])):
    for k in range (len(BestList[i][j])):
      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+1
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Alpha="+str(i)
      NowCell = sheet.cell(row = 2, column = NowColumn)
      NowCell.value = "Beta="+str(j)
      NowCell = sheet.cell(row = 4, column = NowColumn)
      b1=round(BestList[i][j][k][MedianList[i][j][k]][len(BestList[i][j][k][MedianList[i][j][k]])-1],1)
      NowCell.value = "Best="+str(b1)
      
      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+2
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Best"
      flag=0
      if (b1>minDei*1.2):
        flag=1
        t1="NotEnough"
      for Sch1 in range (len(BestList[i][j][k][MedianList[i][j][k]])):
        NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
        NowCell.value =round(BestList[i][j][k][MedianList[i][j][k]][Sch1],1)
        if(flag==0 and BestList[i][j][k][MedianList[i][j][k]][Sch1]<minDei*1.2):
          flag=1
          t1=str(round(TimerList[i][j][k][MedianList[i][j][k]][Sch1],4))
      NowCell = sheet.cell(row = 5, column = NowColumn-1)
      NowCell.value = t1+" сек"

      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+3
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Step"
      for Sch1 in range (len(StepList[i][j][k][MedianList[i][j][k]])):
        NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
        NowCell.value =round(StepList[i][j][k][MedianList[i][j][k]][Sch1])

      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+4
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Time"
      for Sch1 in range (len(TimerList[i][j][k][MedianList[i][j][k]])):
        NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
        NowCell.value =round(TimerList[i][j][k][MedianList[i][j][k]][Sch1],4)


wb.save('Test.xlsx')